In [15]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline
from standard_config import *
import tqdm

In [3]:
haci_bbox_path = "{}/HACI_bbox_data".format(local_data_path)
haci_track_path = "{}/HACI_track_data".format(local_data_path)
linet_grid_path = "{}/linet_grid".format(local_data_path)

In [4]:
haci_cases = pd.read_csv("{}/haci-2008-2017-bbox-filtered.csv".format(haci_bbox_path))
filtered_cases = pd.read_csv("{}/selected_cases.csv".format(haci_bbox_path))

Um den Aufwand etwas geringer zu halten sehen wir uns nur die Jahre 2012 bis 2014 an.

In [5]:
haci_cases = haci_cases[(haci_cases.year >= 2012) & (haci_cases.year <= 2014)]

Das sind immer noch {{len(haci_cases.year)}} Fälle.

In [7]:
haci_dates = haci_cases.date.unique()

In [8]:
haci_dates

array(['2012-01-02', '2012-01-03', '2012-01-05', '2012-01-06',
       '2012-01-07', '2012-01-08', '2012-01-12', '2012-01-13',
       '2012-01-19', '2012-01-23', '2012-01-24', '2012-02-19',
       '2012-02-28', '2012-03-08', '2012-03-17', '2012-03-24',
       '2012-03-29', '2012-04-11', '2012-04-12', '2012-04-19',
       '2012-04-20', '2012-04-21', '2012-04-22', '2012-04-23',
       '2012-04-24', '2012-04-29', '2012-04-30', '2012-05-01',
       '2012-05-04', '2012-05-05', '2012-05-06', '2012-05-09',
       '2012-05-10', '2012-05-11', '2012-05-12', '2012-05-15',
       '2012-05-16', '2012-05-19', '2012-05-20', '2012-05-21',
       '2012-05-22', '2012-05-23', '2012-05-24', '2012-05-25',
       '2012-05-26', '2012-05-27', '2012-05-28', '2012-05-31',
       '2012-06-02', '2012-06-03', '2012-06-04', '2012-06-06',
       '2012-06-07', '2012-06-08', '2012-06-09', '2012-06-11',
       '2012-06-12', '2012-06-13', '2012-06-14', '2012-06-15',
       '2012-06-16', '2012-06-17', '2012-06-18', '2012-

In [62]:
lightning = []

In [10]:
d = haci_dates[0]

In [11]:
haci_cases[haci_cases.date == d]

,date,time,id,flags,t0,t1,l00,l01,c00,c01,...,area_ratio,msg_eu_l00,msg_eu_c00,msg_eu_l01,msg_eu_c01,msg_eu_l0,msg_eu_c0,msg_eu_l1,msg_eu_c1,year
4409,2012-01-02,22:10,8884,0,266,273,156,163,493,504,...,0.235294,213,503,212,506,213,503,211,511,2012


In [63]:
for date in tqdm.tqdm(haci_cases.date.unique()):
    date_cases = haci_cases[haci_cases.date==date]
    
    with xr.open_dataset("{}/linet_radolan_grid_{}.nc".format(linet_grid_path, date.replace("-",""))) as file:
        fld = file.flash_density.data.copy()
        
    for ic, case in date_cases.iterrows():
        tlist = np.arange(np.clip(case.t0,0,288),np.clip(case.t1,0,288),1)
        
        f = []
        
        for t in tlist:
            f.append(fld[t][case.l0:case.l1,case.c0:case.c1])
            
        f_sum = np.sum(np.dstack(f),axis=2)
        
        if np.any(f_sum>0):
            lightning.append(1)
        else:
            lightning.append(0)

100%|██████████| 480/480 [44:32<00:00,  5.57s/it]


In [67]:
haci_cases = haci_cases.assign(lightning=lightning)

In [68]:
haci_cases.to_csv("{}/haci-2012-2014-bbox-filtered.csv".format(haci_bbox_path),index=False)

In [70]:
np.unique(haci_cases.lightning,return_counts=True)

(array([0, 1]), array([1986, 2074]))

In [17]:
dc = haci_cases[haci_cases.date == d].loc[4409]

In [23]:
case =dc

In [34]:
tlist = np.arange(np.clip(case.t0,0,288),np.clip(case.t1,0,288)+6,1)
        
f = []

for t in tlist:
    f.append(fld[t][case.l0:case.l1,case.c0:case.c1])

f_sum = np.sum(np.dstack(f),axis=2)

In [35]:
f_sum


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0

In [36]:
np.any(f_sum>0)

False

In [22]:
np.arange(dc.t0,dc.t1)

array([266, 267, 268, 269, 270, 271, 272])

In [27]:
with xr.open_dataset("{}/linet_radolan_grid_{}.nc".format(linet_grid_path, d.replace("-",""))) as file:
    fld = file.flash_density.data.copy()

In [14]:
fld[haci]

<xarray.DataArray 'flash_density' (time: 288, x: 900, y: 900)>
array([[[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       ...,

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2012-01-02 ... 2012-01-02T23:55:00
    lon      (x, y) float32 3.58893 3.6010756 3.6132221 ... 15.689065 15.704155
    lat      (x, y) float32 46.95258 46.95351 46.954437 ... 54.733574 54.732704
Dimensions without coordinates: x, y
Attributes:
    units:    fl/kilometer^2